<a href="https://colab.research.google.com/github/AsishGYT/Practice-Codes/blob/main/python_db_connection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from sqlalchemy import create_engine
import pandas as pd

In [5]:
#chinook sample DB - from internet - colab resources

In [9]:
### useful: download and extract chinook sample DB
import urllib.request
import zipfile
from functools import partial
import os

chinook_url = 'http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip'
if not os.path.exists('chinook.zip'):
    print('downloading chinook.zip ', end='')
    with urllib.request.urlopen(chinook_url) as response:
        with open('chinook.zip', 'wb') as f:
            for data in iter(partial(response.read, 4*1024), b''):
                print('.', end='', flush=True)
                f.write(data)

zipfile.ZipFile('chinook.zip').extractall()
assert os.path.exists('chinook.db')

downloading chinook.zip ...........................................................................

In [10]:
#helper methods

In [11]:
from IPython.display import display
import pandas as pd

def sql(query):
    print()
    print(query)
    print()

def get_results(query):
    global engine
    q = query.statement if isinstance(query, sqlalchemy.orm.query.Query) else query
    return pd.read_sql(q, engine)

def display_results(query):
    df = get_results(query)
    display(df)
    sql(query)

In [12]:
#open db

In [13]:
import sqlalchemy
from sqlalchemy import create_engine
engine = create_engine('sqlite:///chinook.db')
cur = engine.connect()

In [14]:
### useful: extract classes from the chinook database
metadata = sqlalchemy.MetaData()
metadata.reflect(engine)

## we need to do this once
from sqlalchemy.ext.automap import automap_base

# produce a set of mappings from this MetaData.
Base = automap_base(metadata=metadata)

# calling prepare() just sets up mapped classes and relationships.
Base.prepare()

# also prepare an orm session
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

In [15]:
engine.table_names()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  """Entry point for launching an IPython kernel.


['albums',
 'artists',
 'customers',
 'employees',
 'genres',
 'invoice_items',
 'invoices',
 'media_types',
 'playlist_track',
 'playlists',
 'sqlite_sequence',
 'sqlite_stat1',
 'tracks']

In [17]:
#1st 5 tracks

from sqlalchemy import select 
tracks = Base.classes['tracks']

# using expressions
query = select([tracks]).limit(5)
display_results(query)


,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99



SELECT tracks."TrackId", tracks."Name", tracks."AlbumId", tracks."MediaTypeId", tracks."GenreId", tracks."Composer", tracks."Milliseconds", tracks."Bytes", tracks."UnitPrice" 
FROM tracks
 LIMIT :param_1



In [18]:
# using orm
results = session.query(tracks).limit(5)
display_results(results)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99



SELECT tracks."TrackId" AS "tracks_TrackId", tracks."Name" AS "tracks_Name", tracks."AlbumId" AS "tracks_AlbumId", tracks."MediaTypeId" AS "tracks_MediaTypeId", tracks."GenreId" AS "tracks_GenreId", tracks."Composer" AS "tracks_Composer", tracks."Milliseconds" AS "tracks_Milliseconds", tracks."Bytes" AS "tracks_Bytes", tracks."UnitPrice" AS "tracks_UnitPrice" 
FROM tracks
 LIMIT ? OFFSET ?



In [19]:
from sqlalchemy import join
albums = Base.classes['albums']

# using expression language
query = select([tracks.Name, albums]).select_from(join(tracks, albums)).limit(20)
display_results(query)


,Name,AlbumId,Title,ArtistId
0,For Those About To Rock (We Salute You),1,For Those About To Rock We Salute You,1
1,Put The Finger On You,1,For Those About To Rock We Salute You,1
2,Let's Get It Up,1,For Those About To Rock We Salute You,1
3,Inject The Venom,1,For Those About To Rock We Salute You,1
4,Snowballed,1,For Those About To Rock We Salute You,1
5,Evil Walks,1,For Those About To Rock We Salute You,1
6,C.O.D.,1,For Those About To Rock We Salute You,1
7,Breaking The Rules,1,For Those About To Rock We Salute You,1
8,Night Of The Long Knives,1,For Those About To Rock We Salute You,1
9,Spellbound,1,For Those About To Rock We Salute You,1



SELECT tracks."Name", albums."AlbumId", albums."Title", albums."ArtistId" 
FROM tracks JOIN albums ON albums."AlbumId" = tracks."AlbumId"
 LIMIT :param_1



In [20]:
# using orm
results = session.query(tracks.Name, albums).select_from(tracks).join(albums).limit(20)
display_results(results)

,Name,AlbumId,Title,ArtistId
0,For Those About To Rock (We Salute You),1,For Those About To Rock We Salute You,1
1,Put The Finger On You,1,For Those About To Rock We Salute You,1
2,Let's Get It Up,1,For Those About To Rock We Salute You,1
3,Inject The Venom,1,For Those About To Rock We Salute You,1
4,Snowballed,1,For Those About To Rock We Salute You,1
5,Evil Walks,1,For Those About To Rock We Salute You,1
6,C.O.D.,1,For Those About To Rock We Salute You,1
7,Breaking The Rules,1,For Those About To Rock We Salute You,1
8,Night Of The Long Knives,1,For Those About To Rock We Salute You,1
9,Spellbound,1,For Those About To Rock We Salute You,1



SELECT tracks."Name" AS "tracks_Name", albums."AlbumId" AS "albums_AlbumId", albums."Title" AS "albums_Title", albums."ArtistId" AS "albums_ArtistId" 
FROM tracks JOIN albums ON albums."AlbumId" = tracks."AlbumId"
 LIMIT ? OFFSET ?

